In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

**EBNF:**  
``sequence = letter { letter }``  
``letter = (a | b)``

In [2]:
class Parser1 :
    def __init__(self, source) :
        self.source = source + '$'
        self.sourcePos = 0
    
    def error(self,message) :
        print(message)
        assert False
        
    # sequence = letter { letter }
    def sequence(self) :
        print("sequence",self.sourcePos,self.lookahead)
        self.letter()
        while self.lookahead in {'a','b'} : 
            self.letter()
        if self.lookahead != '$' :
            self.error('$ expected, found ' + str(self.lookahead))
        else :
            print('Success!')

    # letter = (a | b)
    def letter(self) :
        print("letter",self.sourcePos,self.lookahead)
        if self.lookahead in {'a','b'} :
            self.sourcePos = self.sourcePos + 1
            self.lookahead = self.source[self.sourcePos]
        else :
            self.error('a or b expected, found ' + str(self.lookahead))

    def parse(self) :
        try:
            self.sourcePos = 0
            self.lookahead = self.source[self.sourcePos]
            self.sequence()
        except AssertionError :
            print("Aborted!")


In [3]:
p = Parser1("aabaaba")
p.parse()

sequence 0 a
letter 0 a
letter 1 a
letter 2 b
letter 3 a
letter 4 a
letter 5 b
letter 6 a
Success!


**EBNF:**  
``expression = term  { ["+" | "-"] term} ;``  
``term       = factor  { ["*"|"/"] factor} ;``  
``factor     = ident | number | "("  expression  ")" ;``  
``ident	  = "x" ;``  
``constant   = "5" ;``

In [4]:
class Parser2 :
    def __init__(self, source) :
        self.source = source + '$'
        self.sourcePos = 0
    
    def error(self,message) :
        print('Error at position', self.sourcePos)
        print(message)
        assert False
    
    def scan(self) :
        self.sourcePos = self.sourcePos + 1
        self.lookahead = self.source[self.sourcePos]
        
    def expect(self,symbolSet) : 
        if self.lookahead in symbolSet :
            if self.lookahead != '$' : self.scan()
        else :
            self.error(str(symbolSet) + ' expected, found ' + str(self.lookahead))

    # expression = term  { ["+" | "-"] term} ;
    def expression(self) :
        self.term()
        while self.lookahead in {'+','-'} : 
            self.scan()
            self.term()

    # term = factor  { ["*"|"/"] factor} ;
    def term(self) :
        self.factor()
        while self.lookahead in {'*','/'} : 
            self.scan()
            self.factor()
        

    # factor = ident | number | "("  expression  ")" ;
    def factor(self) :
        if self.lookahead == 'x' :
            self.ident()
            return
        if self.lookahead == '5' :
            self.number()
            return
        if self.lookahead == "(" :
            self.scan()
            self.expression()
            self.expect({')'})
            return
        self.error('{x,5,(} expected, found ' + str(self.lookahead))
    
    # ident = "x" ;   
    def ident(self) :
        self.expect({'x'})
    
    # number = "5" ;
    def number(self) :
        self.expect({'5'})   
    
    def parse(self) :
        try:
            # Prologue: Scan first symbol before descent
            self.sourcePos = 0
            self.lookahead = self.source[self.sourcePos]
            # Call start symbol
            self.expression()
            # Epilogue: Check for EOF
            self.expect({'$'})
            print('Success!')
        except AssertionError :
            print("Aborted!")

In [10]:
p = Parser2("5*(5-5)*x-5*5")
p.parse()

Success!
